In [1]:
import qutip as qt
import numpy as np
print(qt.__version__)


5.0.4


## First we define state and density matrix

In [2]:

def create_random_state(n):
    basis_states = [qt.basis([2]*n, [int(i) for i in format(j, f'0{n}b')]) for j in range(2**n)]
    
    random_probs = np.random.rand(2**n)
    random_probs /= np.sum(random_probs)  # Normalize to sum to 1
    
    coefficients = np.sqrt(random_probs)
   
    random_state = sum(coeff * basis for coeff, basis in zip(coefficients, basis_states))
    return random_state

def compute_density_matrix(state):
    return state * state.dag()

num_qubits = 3  

random_state_1 = create_random_state(num_qubits)
random_state_2 = create_random_state(num_qubits)
random_state_3 = create_random_state(num_qubits)

p_random_1 = compute_density_matrix(random_state_1)
p_random_2 = compute_density_matrix(random_state_2)
p_random_3 = compute_density_matrix(random_state_3)

p_mixed = (1/3) * (p_random_1 + p_random_2 + p_random_3)


np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("Density Matrix in Matrix Form:")
print(np.real(p_mixed.full()))



Density Matrix in Matrix Form:
[[0.109 0.119 0.117 0.091 0.094 0.113 0.104 0.119]
 [0.119 0.145 0.14  0.124 0.109 0.13  0.116 0.144]
 [0.117 0.14  0.147 0.125 0.113 0.141 0.103 0.144]
 [0.091 0.124 0.125 0.119 0.094 0.114 0.086 0.125]
 [0.094 0.109 0.113 0.094 0.087 0.108 0.085 0.112]
 [0.113 0.13  0.141 0.114 0.108 0.138 0.097 0.136]
 [0.104 0.116 0.103 0.086 0.085 0.097 0.108 0.112]
 [0.119 0.144 0.144 0.125 0.112 0.136 0.112 0.145]]


## Conditional Mutual Information bound check

In [19]:
def compute_reduced_density_matrices(p_W):
    p_A = p_mixed.ptrace([0])  # Tracing out B and C
    p_B = p_mixed.ptrace([1])  # Tracing out A and C
    p_C = p_mixed.ptrace([2])  # Tracing out A and B
    p_AB = p_mixed.ptrace([0, 1])  # Tracing out C
    p_AC = p_mixed.ptrace([0, 2])  # Tracing out B
    p_BC = p_mixed.ptrace([1, 2])  # Tracing out A
    return p_A, p_B, p_C, p_AB, p_AC, p_BC


p_A, p_B, p_C, p_AB, p_AC, p_BC = compute_reduced_density_matrices(p_mixed)

def compute_entropies(p_A, p_B, p_C, p_AB, p_AC, p_BC, p_W):
    S_A = qt.entropy_vn(p_A)
    S_B = qt.entropy_vn(p_B)
    S_C = qt.entropy_vn(p_C)
    S_AB = qt.entropy_vn(p_AB)
    S_AC = qt.entropy_vn(p_AC)
    S_BC = qt.entropy_vn(p_BC)
    S_W = qt.entropy_vn(p_W)  # Total system entropy
    return S_A, S_B, S_C, S_AB, S_AC, S_BC, S_W

S_A, S_B, S_C, S_AB, S_AC, S_BC, S_mixed = compute_entropies(p_A, p_B, p_C, p_AB, p_AC, p_BC, p_mixed)


In [21]:
# Nicely formatted printout
entropy_labels = ["S(ρ_A)", "S(ρ_B)", "S(ρ_C)", "S(ρ_AB)", "S(ρ_AC)", "S(ρ_BC)", "S(ρ_mixed)"]
entropy_values = [S_A, S_B, S_C, S_AB, S_AC, S_BC, S_mixed]

# Print header
print("=" * 40)
print(f"{'Subsystem':<10} | {'Entropy':>12}")
print("=" * 40)

# Print entropy values
for label, value in zip(entropy_labels, entropy_values):
    print(f"{label:<10} | {value:12.6f}")

print("=" * 40)


Subsystem  |      Entropy
S(ρ_A)     |     0.190994
S(ρ_B)     |     0.161801
S(ρ_C)     |     0.145843
S(ρ_AB)    |     0.259690
S(ρ_AC)    |     0.280071
S(ρ_BC)    |     0.286711
S(ρ_mixed) |     0.271307


## Compute the Conditional Mutual Information

In [27]:
I_ABcC = S_AC + S_BC - S_mixed - S_C
print("Conditional mutual information I(A:B|C) =",I_ABcC)

Conditional mutual information I(A:B|C) = 0.14963102178054116


## Compute 3 possible bounds using Z operator

In [23]:
Z = qt.sigmaz()
Z_2 = qt.tensor(Z, Z)
Z_full = qt.tensor(Z, Z, Z)


avg_Z_A = (p_A * Z).tr()
avg_Z_B = (p_B * Z).tr()
avg_Z_C = (p_C * Z).tr()
avg_Z_AC = (p_AC * Z_2).tr()
avg_Z_BC = (p_BC * Z_2).tr()
avg_Z_AB = (p_AB * Z_2).tr()
avg_Z_full = (p_mixed * Z_full).tr()

Z_norm = 1



In [24]:
# Define labels and values
z_labels = [
    "⟨Z_A⟩", "⟨Z_B⟩", "⟨Z_C⟩",
    "⟨Z_AC⟩", "⟨Z_BC⟩", "⟨Z_AB⟩",
    "⟨Z_W⟩ (Full)"
]
z_values = [
    avg_Z_A, avg_Z_B, avg_Z_C,
    avg_Z_AC, avg_Z_BC, avg_Z_AB,
    avg_Z_full
]

# Print header
print("=" * 40)
print(f"{'Observable':<10} | {'Expectation Value':>20}")
print("=" * 40)

# Print expectation values
for label, value in zip(z_labels, z_values):
    print(f"{label:<10} | {value:20.6f}")

print("=" * 40)


Observable |    Expectation Value
⟨Z_A⟩      |   0.042221+0.000000j
⟨Z_B⟩      |  -0.039808+0.000000j
⟨Z_C⟩      |  -0.095856+0.000000j
⟨Z_AC⟩     |   0.081358+0.000000j
⟨Z_BC⟩     |  -0.077151+0.000000j
⟨Z_AB⟩     |   0.015454+0.000000j
⟨Z_W⟩ (Full) |  -0.050608+0.000000j


In [35]:
f_1 = 0.5*(((avg_Z_full-(avg_Z_A*avg_Z_BC))**2 + (avg_Z_full-(avg_Z_B*avg_Z_AC))**2)/(2*3*Z_norm**2)-2)
f_2 = (avg_Z_full-avg_Z_A*avg_Z_B*avg_Z_C)**2/(2*3*Z_norm**2)-2
f_3 = (np.real(avg_Z_full)-np.real(avg_Z_A)*np.real(avg_Z_B))**2/(2*2*Z_norm**2) -3
f_4 = (avg_Z_full*avg_Z_C - avg_Z_AC*avg_Z_BC)**2/(2*4*Z_norm**2)

print(" f_1 = ", np.real(f_1),"\n f_2 = ", np.real(f_2),"\n f_3 =",np.real(f_3),"\n f_4 = ", np.real(f_4))

 f_1 =  -0.9996261750323719 
 f_2 =  -1.9995704193951211 
 f_3 = -2.999401534385634 
 f_4 =  1.547898872158488e-05
